In [155]:
##################################################################
# Establishing Equivalence between Scopus and ABDC Journal lists #
##################################################################
import pandas as pd

directory = '/Users/kalyan/OneDrive-IIMV/OneDrive - IIM VIZAG/IIM Vizag/IIMV-AdminWork/ABDC-Scopus-Equivalence/'

# Import ABDC MASTER FILE available @ https://abdc.edu.au/research/abdc-journal-quality-list/

abdc_master = pd.read_excel('/Users/kalyan/OneDrive-IIMV/OneDrive - IIM VIZAG/IIM Vizag/IIMV-AdminWork/ABDC-Scopus-Equivalence/abdc_jql_2019_v12.xlsx', 
                            header=8,dtype={'ISSN':str,'ISSN Online':str})
#abdc_master = pd.read_excel(directory/ 'abdc_jql_2019_v12.xlsx', header=8)
abdc_master['NEWISSN'] = abdc_master['ISSN'].str[:4] + abdc_master['ISSN'].str[-5:]
abdc_master['NEWISSNONLINE'] = abdc_master['ISSN Online'].str[:4] + abdc_master['ISSN Online'].str[-5:-1]
abdc_master['NEWISSN']=abdc_master['NEWISSN'].str.replace("\t","")
abdc_master['NEWISSNONLINE'] =abdc_master['NEWISSNONLINE'].str.replace("\t","")
abdc_master[['NEWISSN', 'NEWISSNONLINE']]
print(abdc_master.columns)

# Import SCIMAGO JR MASTER FILE available @ https://www.scimagojr.com/journalrank.php
scimagojr_master = pd.read_csv('/Users/kalyan/OneDrive-IIMV/OneDrive - IIM VIZAG/IIM Vizag/IIMV-AdminWork/ABDC-Scopus-Equivalence/scimagojr 2020.csv', 
                            sep=';', dtype={'Issn':str})
new = scimagojr_master['Issn'].str.split(",",n=1, expand=True)
scimagojr_master['NEWISSN_1']= new[0]
scimagojr_master['NEWISSN_2']= new[1]
scimagojr_master['Categories_Backup']=scimagojr_master['Categories']
scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q1\)","")
scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q2\)","")
scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q3\)","")
scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q4\)","")
# Strip whitespaces before and after string
scimagojr_master['Categories']=scimagojr_master['Categories'].str.strip()
#scimagojr_master['Categories']=scimagojr_master['Categories'].str.lstrip()

tmp=scimagojr_master['Categories'].str.split()
print(tmp)


# Code to identify and split as many categories a journal belongs to.
# First identify maximum number of categories
scimagojr_master['Num_Categories']=scimagojr_master.Categories.str.count(";")
print("Maximum num of categories:",scimagojr_master.Num_Categories.max())
cat = scimagojr_master['Categories'].str.split(";",n=scimagojr_master.Num_Categories.max(), expand=True)


# create one column each for categories of a journal
i = 0
while i<scimagojr_master.Num_Categories.max():
    scimagojr_master['Category_{}'.format(i+1)]= cat[i]
    #print(i)
    i=i+1
    
print(scopus_not_in_abdc['Category_1'].unique())

#Boolean filter: Look for exclusively SCOPUS journals (which are not available in ABDC list)
scopus_not_in_abdc = scimagojr_master[~(scimagojr_master.NEWISSN_1.isin(abdc_master['NEWISSN']) 
                                        | 
                                        scimagojr_master.NEWISSN_2.isin(abdc_master['NEWISSN']))]
print(scimagojr_master.shape[0],scopus_not_in_abdc.shape[0])
print(scopus_not_in_abdc.columns)
pd.crosstab(scopus_not_in_abdc['Category_1']=='Economics and Econometrics',scopus_not_in_abdc['SJR Best Quartile'])

print(scopus_not_in_abdc[scopus_not_in_abdc['Category_1']=='Economics and Econometrics'][['Title', 'Issn']])


Index(['Journal Title', 'Publisher', 'ISSN', 'ISSN Online', 'Year Inception',
       'Field of Research', '2019 Rating', 'NEWISSN', 'NEWISSNONLINE'],
      dtype='object')


/Users/kalyan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-155-72ad8100c59d>:27: FutureWarning: The default value of regex will change from True to False in a future version.
  scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q1\)","")
<ipython-input-155-72ad8100c59d>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q2\)","")
<ipython-input-155-72ad8100c59d>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  scimagojr_master['Categories']=scimagojr_master['Categories'].str.replace("\(Q3\)","")
<ipython-input-155-72ad8100c59d>:30: FutureWarning: The default value of regex will change fro

0                                [Hematology, ;, Oncology]
1        [Epidemiology, ;, Health, Information, Managem...
2                   [Cell, Biology, ;, Molecular, Biology]
3                           [Economics, and, Econometrics]
4        [Biomaterials, ;, Electronic,, Optical, and, M...
                               ...                        
32947                          [Medicine, (miscellaneous)]
32948                          [Medicine, (miscellaneous)]
32949                          [Medicine, (miscellaneous)]
32950                          [Medicine, (miscellaneous)]
32951    [Immunology;, Medicine, (miscellaneous);, Micr...
Name: Categories, Length: 32952, dtype: object
Maximum num of categories: 12


KeyError: 'Category_1'

In [10]:
##################################################################
# Establishing Equivalence between Scopus and ABDC Journal lists #
##################################################################
import pandas as pd
import openpyxl as opyxl
dir = '/Users/kalyan/OneDrive-IIMV/OneDrive - IIM VIZAG/IIM Vizag/IIMV-AdminWork/ABDC-Scopus-Equivalence/'

# Import ABDC MASTER FILE available @ https://abdc.edu.au/research/abdc-journal-quality-list/
abdc_master = pd.read_excel('{}/abdc_jql_2019_v12.xlsx'.format(dir),header=8,dtype={'ISSN':str,'ISSN Online':str})
abdc_master['NEWISSN'] = abdc_master['ISSN'].str[:4] + abdc_master['ISSN'].str[-5:]
abdc_master['NEWISSNONLINE'] = abdc_master['ISSN Online'].str[:4] + abdc_master['ISSN Online'].str[-5:-1]
abdc_master['NEWISSN']=abdc_master['NEWISSN'].str.replace("\t","")
abdc_master['NEWISSNONLINE'] =abdc_master['NEWISSNONLINE'].str.replace("\t","")
#abdc_master[['NEWISSN', 'NEWISSNONLINE']]
print(abdc_master.columns)

# Import SCOPUS SOURCE LIST MASTER FILE available @ https://www.scopus.com/sources.uri?zone=TopNavBar&origin=searchbasic
scopus_master = pd.read_excel('{}/CiteScore 2011-2020 new methodology - May 2021 2.xlsx'.format(dir), 
                            sheet_name='CiteScore 2020', dtype={'Print ISSN':str, 'E-ISSN':str})
scopus_master['Print-ISSN']=scopus_master['Print ISSN']
print(scopus_master.columns)
#Boolean filter: Look for exclusively SCOPUS journals (which are not available in ABDC list)
scopus_not_in_abdc = scopus_master[~(
                                    scopus_master['Print-ISSN'].isin(abdc_master['NEWISSN']) 
                                        | 
                                    scopus_master['E-ISSN'].isin(abdc_master['NEWISSNONLINE'])
                                    )]
# Generate Decile information based on SJR score
scopus_not_in_abdc['SJR_Deciles'] = pd.qcut(scopus_not_in_abdc.SJR, q=10,labels=False)
# Create tabulations of number of titles within each decile based on SJR score
new_df = pd.DataFrame(pd.crosstab(scopus_not_in_abdc['Scopus Sub-Subject Area'],scopus_not_in_abdc['SJR_Deciles']))

df2=scopus_not_in_abdc[scopus_not_in_abdc['Scopus Sub-Subject Area']=='Economics and Econometrics'][['Title','SJR','SJR_Deciles','Quartile']]

# Write to Excel file
path = '{}/SCOPUS-ABDC-EQUIVALENCE.xlsx'.format(dir)
with pd.ExcelWriter(path) as writer:
    new_df.to_excel(writer, sheet_name='SJR_DECILES')
    df2.to_excel(writer, sheet_name='Econ')
#### END ####

Index(['Journal Title', 'Publisher', 'ISSN', 'ISSN Online', 'Year Inception',
       'Field of Research', '2019 Rating', 'NEWISSN', 'NEWISSNONLINE'],
      dtype='object')
Index(['Scopus Source ID', 'Title', 'Citation Count', 'Scholarly Output',
       'Percent Cited', 'CiteScore 2020', 'SNIP', 'SJR',
       'Scopus ASJC Code (Sub-subject Area)', 'Scopus Sub-Subject Area',
       'Percentile', 'RANK', 'Rank Out Of', 'Publisher', 'Type', 'Open Access',
       'Quartile', 'Top 10% (CiteScore Percentile)', 'URL Scopus Source ID',
       'Print ISSN', 'E-ISSN', 'Print-ISSN'],
      dtype='object')


<ipython-input-10-1bae4b6bc542>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scopus_not_in_abdc['SJR_Deciles'] = pd.qcut(scopus_not_in_abdc.SJR, q=10,labels=False)
